In [1]:
sc

''

In [2]:
sc

''

In [1]:
sc

''

''

In [2]:
from pyspark import SparkContext

AttributeError: module 'numpy' has no attribute 'core'

In [3]:
import pyspark

AttributeError: module 'numpy' has no attribute 'core'

In [1]:
sc